# 实验 v1.0

## 思路

使用【可解释方法】对实验数据【预处理】消除歧视样本，并【重新训练】，目的是提升【模型的公平性】

## 实验简介

1. 选择数据集 --> Adult
2. 选择训练模型 --> XGboost
3. 选择可解释模型 --> Shap
4. 选择不公平的数据样本 --> 选择敏感属性的 |shap_values| 的较大值
5. 选择 k1 个最不公平的数据样本 --> k1=1000 约占训练集的 5%
6. 构造 k2 个预处理样本呢 --> 令 k2=k1
7. 选择预处理策略 --> 更改敏感属性标签并加入训练集中
8. 选择公平性指标 --> dp 和 eo

## 如何优化

1. 如何使用可解释的输出来选择不公平的数据样本
2. 选择多少个样本（百分比）
3. 构造样本时，是保持不变（`k1=k2`）还是膨胀（`k1<k2`）
4. 如何选择预处理策略，直接加入后，X_train 是否要重新缩放
5. 如何选择公平性指标


## 实验基本环境设置

In [1]:
# 导入依赖
import shap
import numpy as np
import pandas as pd


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# 字体设置
from pylab import mpl
# 设置显示中文字体
mpl.rcParams["font.sans-serif"] = ["SimHei"]

## 加载原始数据

In [5]:
# 加载数据
file = ('../input/adult.csv')
df = pd.read_csv(file, encoding='latin-1')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


## 数据预处理部分

In [4]:
# 数据预处理
df[df == '?'] = np.nan
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)
X = df.drop(['income'], axis=1)
y = df['income']
y = y.map({"<=50K":0, ">50K":1})
# y

In [101]:
# 函数：数据拆分
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def split_df(X, y):
    # 分割数据集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0
    )

    # 将标签数字化
    categorical = [
        "workclass",
        "education",
        "marital.status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native.country",
    ]
    for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

    # 数据缩放
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    return X_train, X_test, y_train, y_test

In [102]:
# 数据拆分
X_train, X_test, y_train, y_test = split_df(X, y)

In [103]:
# 设置敏感属性
X_test["sex"].value_counts()
sex = X_test['sex']
sex = sex.map(lambda x : "男" if x > 0 else "女")

In [104]:
# 获得敏感属性的索引
feature_index = X.columns.get_loc("sex")
feature_index

9

## 指标预设

In [105]:
# 普通指标和公平性指标
from sklearn.metrics import accuracy_score, precision_score


from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
    mean_prediction,

)


from fairlearn.metrics import (
    false_positive_rate,
    false_negative_rate,
    count,
    selection_rate,

    MetricFrame,
)



def test_metrics(y_true, y_pred, sensitive_features):
    normal_metrics = {

        "样本数      ": count,
        "准确率      ": accuracy_score,  # 越大越好
        "精确率      ": precision_score,  # 越大越好
        "FPR        ": false_positive_rate,  # 越小越好
        "FNR        ": false_negative_rate,  # 越小越好
        # "选择率      ": selection_rate,
        # "公平性平均预测": mean_prediction,  # 衡量不同群体之间的平均预测值是否相等
    }

    fairness_metrics = {
        "人口平等 差异": demographic_parity_difference,  # 越小越好
        "人口平等 比率": demographic_parity_ratio,  # 越接近 1 越好
        "机会均等 差异": equalized_odds_difference,  # 越小越好
        "机会均等 比率": equalized_odds_ratio,  # 越接近 1 越好
    }

    metric_frame = MetricFrame(
        metrics=normal_metrics,
        y_true=y_true,
        y_pred=y_pred,
        sensitive_features=sensitive_features,
    )
    
    # # 画图
    # metric_frame.by_group.plot.bar(
    #     subplots=True,
    #     layout=[3, 3],
    #     legend=False,
    #     figsize=[12, 8],

    #     title="普通指标",
    # )

    print(metric_frame.overall)

    params = {
        "y_true": y_true,
        "y_pred": y_pred,
        "sensitive_features": sensitive_features,
    }

    for k, v in fairness_metrics.items():
        print(k, v(**params))

## 模型：逻辑回归

In [106]:
# 逻辑回归 - 训练
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [107]:
# 逻辑回归 - 预测
y_pred = logreg.predict(X_test)

In [108]:
# 逻辑回归 - 指标
test_metrics(y_test, y_pred, sex)

样本数            9769.000000
准确率               0.821681
精确率               0.710868
FPR               0.057085
FNR               0.559135
dtype: float64
人口平等 差异 0.17305965639298973
人口平等 比率 0.16808117698325414
机会均等 差异 0.2678519815463287
机会均等 比率 0.15100051877108894


## 模型：XGBoost

In [109]:
# XGboost - 训练
import xgboost
xgboost_model = xgboost.XGBRegressor().fit(X_train, y_train)

In [110]:
# XGboost - 预测
y_pred_xgboost_1 = xgboost_model.predict(X_test)
y_pred_xgboost_1 = np.where(y_pred_xgboost_1 > 0.5, 1, 0)

In [111]:
# XGboost - 指标
test_metrics(y_test, y_pred_xgboost_1, sex)

样本数            9769.000000
准确率               0.862422
精确率               0.768377
FPR               0.059109
FNR               0.384061
dtype: float64
人口平等 差异 0.16989595884523423
人口平等 比率 0.32250719180485066
机会均等 差异 0.08764799066199769
机会均等 比率 0.29597011190360795


## 使用可解释工具进行分析

In [112]:
# 计算 shap_values
explainer = shap.Explainer(xgboost_model)
shap_values = explainer(X_train)

In [113]:
# 获得 k 个最大的索引
arr = shap_values[:, feature_index].values # numpy.ndarray
sorted_indices = np.argsort(np.abs(arr))
k = 1000
top_k_indices = sorted_indices[-k:]
# top_k_indices

In [114]:
# 获得 X_ext 拓展
X_ext = X_train.iloc[top_k_indices, :]
y_ext = y_train.iloc[top_k_indices]

In [115]:
# 统计缩放后的个数
tmp = X_train["sex"].unique()
tmp
X_train["sex"].value_counts()

sex
 0.699071    15310
-1.430470     7482
Name: count, dtype: int64

In [116]:
# 敏感属性 反转
X_ext.loc[:, "sex"] = np.where(X_ext.loc[:, "sex"] < 0, tmp[1], tmp[0])
# X_ext.head()

In [117]:
# 索引连接
# print(X_train.shape, X_test.shape)
X_train_2 = pd.concat([X_train, X_ext])
y_train_2 = pd.concat([y_train, y_ext])

## 模型：XGBoost + 处理过的训练集

In [118]:
# XGboost_2 - 训练
import xgboost
xgboost_model_2 = xgboost.XGBRegressor().fit(X_train_2, y_train_2)

In [119]:
# XGboost_2 - 预测
y_pred_xgboost_2 = xgboost_model_2.predict(X_test)
y_pred_xgboost_2 = np.where(y_pred_xgboost_2 > 0.5, 1, 0)

In [120]:
# # XGboost_2 - 指标
test_metrics(y_test, y_pred_xgboost_2, sex)

样本数            9769.000000
准确率               0.864162
精确率               0.770724
FPR               0.058974
FNR               0.377279
dtype: float64
人口平等 差异 0.17175240889009005
人口平等 比率 0.3209544785797538
机会均等 差异 0.08581790895447716
机会均等 比率 0.286851314441789


In [122]:
test_metrics(y_test, y_pred_xgboost_1, sex)

样本数            9769.000000
准确率               0.862422
精确率               0.768377
FPR               0.059109
FNR               0.384061
dtype: float64
人口平等 差异 0.16989595884523423
人口平等 比率 0.32250719180485066
机会均等 差异 0.08764799066199769
机会均等 比率 0.29597011190360795
